In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection
engine = create_engine('mssql+pyodbc://LUCAS\\SQLEXPRESS01/Riotdb?driver=ODBC+Driver+17+for+SQL+Server')

# Function to execute stored procedure and return the result as a pandas DataFrame
def execute_proc(proc_name, summoner_name=None):
    # If SummonerName is provided, include it as a parameter in the query
    if summoner_name:
        query = f"EXEC {proc_name} @SummonerName='{summoner_name}'"
    else:
        query = f"EXEC {proc_name}"
    
    return pd.read_sql_query(query, engine)

# Input for SummonerName (since the procedure expects it)
summoner_name = input("Enter the Summoner Name: ")

# Execute stored procedures
df_league_params = execute_proc('Parametros_Detalhes_Liga')
df_summoner_status = execute_proc('Parametros_Detalhes_Invocador', summoner_name=summoner_name)

# Compute the performance score as column 2 / column 1
df_performance_score = df_summoner_status / df_league_params

# Create labels for each metric
labels = df_league_params.columns.tolist()

# Combine the results into one DataFrame, adding the labels as the first column
df_combined = pd.DataFrame({
    'Metric': labels,
    'League Parameters': df_league_params.squeeze(),
    'Summoner Status': df_summoner_status.squeeze(),
    'Performance Score': df_performance_score.squeeze()
})

# Export the DataFrame to Excel
output_file = 'summoner_performance_report.xlsx'
df_combined.to_excel(output_file, index=False)

# Print success message
print(f"Data exported successfully to {output_file}")


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]O procedimento ou a função 'Parametros_Detalhes_Invocador' espera o parâmetro '@riotIdGameName', que não foi fornecido. (201) (SQLExecDirectW)")
[SQL: EXEC Parametros_Detalhes_Invocador @SummonerName='JKaah']
(Background on this error at: https://sqlalche.me/e/20/f405)